### Test to see what was needed to run the planet generator with the saved training file.
#### A version of this code now sits on heroku to power the bot.

In [ ]:
import numpy as np
import itertools
import re
from keras.models import Model, load_model
from keras.preprocessing.text import Tokenizer

In [ ]:
 model = load_model('./astroph_no_dropout_100.h5')

In [ ]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    
    seed_text = start_title + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "|":
            break
            
        output_text += output_word + ' '
        seed_text += output_word + ' '
 
                    
    return output_text

In [ ]:
filename = "./papertitles_text.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()
    
start = 0 
end = len(text) 
text = text[start:end]

tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])

In [ ]:
# Routine to switch generated title from lower-case back to title-like punctuation 
# Basically a bunch of clunky string manipulation.
def capitalise(title):
    
    # words that shouldn't be capitalised unless their the first word in the title
    unimportant=["a", "an", "the", "am" ,"is", "are", "and", "of", "in" , "on", "with", "from", "to", "as", "by"]
    # Acroynms that should be all caps
    capitalize = ["ska", "ngc", "grb", "ii"]

    resp=""
    v = title.split()
    for idx, x in enumerate(v):
        if x not in unimportant:
            if x in capitalize:
                resp += (" " + x.upper())
            else: 
                resp += (" " + ''.join(x.capitalize()))
        else:
            if idx == 0: # first word
                resp += (" " + ''.join(x.capitalize()))
            else:
                resp += (" " + x)

    return resp

print(capitalise("an example of a title. SKA and the galaxy's stars."))

In [ ]:
seed_text =  " "   
gen_words = 35
temp = 0.8
max_seq_length = 10
start_title = '| ' * max_seq_length

text = generate_text(seed_text, gen_words, model, max_seq_length, temp)
print("Original text: ", text)
    
# remove duplicate words, e.g. "the the"
text = ' '.join(c[0] for c in itertools.groupby(text.split()))

#check if in training set (sometimes title is just from the training set.)
# (In the bot, the title is re-generated if it's a duplicate.)
training = open("./astrobotph_twitter/papertitles_list.txt", "r")
training_titles = []
for line in training:
    training_titles.append(line.strip())
    
if text in training_titles:
    print("match: ", text)
else:
    text = capitalise(text)
    

print(text)

In [ ]:
# boxes to experiment with string recognition and reply in the tweet to the bot
buffer = "@astrobotph “planet“"
buffer = "\"planet\""
quotes = re.split('\" |\"|\” |\”|\“ |\“|\” |\”|\“ |\“|\*|\n', buffer)[1:2][0].strip().lower()+" "
quotes

In [ ]:
reply = "\"" + quotes + "\""
reply

In [ ]:
tweet_text = "Congratulations! Your paper " + "\""+text.strip()+ "\"" + " has been accepted to the Journal of AstRobotomy."
tweet_text